In [1]:
import os
import glob
import pandas as pd

import numpy as np


In [2]:
# Define working directory
os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify\Original_data")

In [3]:
# Use glob to match the pattern ‘csv’
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

# Combine all files in the list and export as CSV
df = pd.concat([pd.read_csv(f) for f in all_filenames ])
df.reset_index(drop=True, inplace=True)
df.head(5)

,track,artist,uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,chorus_hit,sections,target
0,Lucky Man,Montgomery Gentry,spotify:track:4GiXBCUF7H6YfNQsnBRIzl,0.578,0.471,4,-7.270,1,0.0289,0.368000,0.00000,0.159,0.532,133.061,196707,4,30.88059,13,1
1,On The Hotline,Pretty Ricky,spotify:track:1zyqZONW985Cs4osz9wlsu,0.704,0.854,10,-5.477,0,0.1830,0.018500,0.00000,0.148,0.688,92.988,242587,4,41.51106,10,1
2,Clouds Of Dementia,Candlemass,spotify:track:6cHZf7RbxXCKwEkgAZT4mY,0.162,0.836,9,-3.009,1,0.0473,0.000111,0.00457,0.174,0.300,86.964,338893,4,65.32887,13,0
3,"Heavy Metal, Raise Hell!",Zwartketterij,spotify:track:2IjBPp2vMeX7LggzRN3iSX,0.188,0.994,4,-3.745,1,0.1660,0.000007,0.07840,0.192,0.333,148.440,255667,4,58.59528,9,0
4,I Got A Feelin',Billy Currington,spotify:track:1tF370eYXUcWwkIvaq3IGz,0.630,0.764,2,-4.353,1,0.0275,0.363000,0.00000,0.125,0.631,112.098,193760,4,22.62384,10,1


In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
# Reformat data
data = df2.values
X = data[:, 0:15]  
y = data[:, 15]

print(data.shape, X.shape, y.shape)
data
# X[0]
# y

(41106, 16) (41106, 15) (41106,)


array([[ 0.578  ,  0.471  ,  4.     , ..., 30.88059, 13.     ,  1.     ],
       [ 0.704  ,  0.854  , 10.     , ..., 41.51106, 10.     ,  1.     ],
       [ 0.162  ,  0.836  ,  9.     , ..., 65.32887, 13.     ,  0.     ],
       ...,
       [ 0.562  ,  0.314  , 10.     , ..., 21.11763, 10.     ,  1.     ],
       [ 0.622  ,  0.781  ,  7.     , ..., 47.13558, 11.     ,  0.     ],
       [ 0.664  ,  0.739  ,  2.     , ..., 42.50341, 14.     ,  1.     ]])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [7]:
#Scale features (X)
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[ 1.35315003, -0.22396167,  1.07008741, ...,  0.24964134,
         0.64353579,  0.1078218 ],
       [-0.03430522,  0.9971179 ,  1.07008741, ...,  0.24964134,
         0.88344544, -0.71419692],
       [-0.2599077 ,  0.34296813,  1.07008741, ...,  0.24964134,
         0.72333873, -0.91970159],
       ...,
       [ 0.16873701, -0.838466  ,  1.07008741, ...,  0.24964134,
         0.07947695,  1.13534519],
       [-0.71111266, -1.65912662,  0.50456543, ...,  0.24964134,
         0.66731336, -0.50869224],
       [-1.39920022, -1.31024674,  1.07008741, ..., -2.13381886,
        -0.45146094,  0.1078218 ]])

One-hot encode the labels

In [8]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Model Architecture

In [9]:
#First create a sequential model
from tensorflow.keras.models import Sequential

model = Sequential() 

In [10]:
#For categorical data, we use a classifier model (Softmax)
from tensorflow.keras.layers import Dense
number_inputs = 15  
number_hidden_nodes = 12

#Create hidden layer
model.add(Dense(units=number_hidden_nodes,activation='relu', input_dim=number_inputs))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes:
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
model.fit(X_train_scaled,y_train_categorical,epochs=100, shuffle=True, verbose=2)

Epoch 1/100
964/964 - 1s - loss: 0.5895 - accuracy: 0.6968
Epoch 2/100
964/964 - 1s - loss: 0.5070 - accuracy: 0.7481
Epoch 3/100
964/964 - 1s - loss: 0.4958 - accuracy: 0.7566
Epoch 4/100
964/964 - 1s - loss: 0.4894 - accuracy: 0.7633
Epoch 5/100
964/964 - 1s - loss: 0.4858 - accuracy: 0.7659
Epoch 6/100
964/964 - 1s - loss: 0.4831 - accuracy: 0.7680
Epoch 7/100
964/964 - 1s - loss: 0.4807 - accuracy: 0.7702
Epoch 8/100
964/964 - 1s - loss: 0.4793 - accuracy: 0.7700
Epoch 9/100
964/964 - 1s - loss: 0.4778 - accuracy: 0.7717
Epoch 10/100
964/964 - 1s - loss: 0.4767 - accuracy: 0.7718
Epoch 11/100
964/964 - 2s - loss: 0.4756 - accuracy: 0.7729
Epoch 12/100
964/964 - 2s - loss: 0.4751 - accuracy: 0.7720
Epoch 13/100
964/964 - 2s - loss: 0.4748 - accuracy: 0.7722
Epoch 14/100
964/964 - 1s - loss: 0.4739 - accuracy: 0.7736
Epoch 15/100
964/964 - 1s - loss: 0.4736 - accuracy: 0.7726
Epoch 16/100
964/964 - 1s - loss: 0.4731 - accuracy: 0.7732
Epoch 17/100
964/964 - 1s - loss: 0.4728 - accura

## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
    
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 0s - loss: 0.4695 - accuracy: 0.7766
Loss: 0.46954256296157837, Accuracy: 0.776588499546051


In [15]:
#Make Predictions
encoded_predictions = model.predict_classes(X_test_scaled[:5])
print(f"Predicted classes: {encoded_predictions}")
print(f"Actual Labels: {list(y_test[:5])}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted classes: [1 1 0 1 1]
Actual Labels: [1.0, 1.0, 1.0, 0.0, 0.0]


## Saving the Trained Model

In [16]:
# # Save the model
# Define working directory
os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify")
model.save("spotify_model.h5")

## Loading a Model

In [17]:
# Load the model  
from tensorflow.keras.models import load_model
# Define working directory
os.chdir(r"C:\Users\hguzm\Documents\000. Personal\Bootcamp\Proyectos\FinalProject-Spotify")
model = load_model("spotify_model.h5")

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 218
Trainable params: 218
Non-trainable params: 0
_________________________________________________________________


In [19]:
#Evaluate the Model using the testing data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
    
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

322/322 - 0s - loss: 0.4695 - accuracy: 0.7766
Loss: 0.46954256296157837, Accuracy: 0.776588499546051


## Deep Learning

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()

number_inputs = 15  
number_hidden_nodes = 12
number_hidden_nodes2 = 10
number_classes = 2

deep_model.add(Dense(units=number_hidden_nodes, activation='relu', input_dim=number_inputs))
deep_model.add(Dense(units=number_hidden_nodes2, activation='relu'))
deep_model.add(Dense(units=number_classes, activation='softmax'))

In [21]:
deep_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                130       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 22        
Total params: 344
Trainable params: 344
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Compile the model
deep_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

#Train the model
deep_model.fit(X_train_scaled, y_train_categorical, epochs=100, shuffle=True, verbose=2)

#Make Predictions
encoded_predictions = model.predict_classes(X_test_scaled[:5])
print(f"Predicted classes: {encoded_predictions}")
print(f"Actual Labels: {list(y_test[:5])}")

Epoch 1/100
964/964 - 1s - loss: 0.5585 - accuracy: 0.7075
Epoch 2/100
964/964 - 1s - loss: 0.5007 - accuracy: 0.7511
Epoch 3/100
964/964 - 1s - loss: 0.4923 - accuracy: 0.7582
Epoch 4/100
964/964 - 1s - loss: 0.4868 - accuracy: 0.7613
Epoch 5/100
964/964 - 1s - loss: 0.4826 - accuracy: 0.7658
Epoch 6/100
964/964 - 1s - loss: 0.4791 - accuracy: 0.7700
Epoch 7/100
964/964 - 1s - loss: 0.4764 - accuracy: 0.7709
Epoch 8/100
964/964 - 1s - loss: 0.4739 - accuracy: 0.7738
Epoch 9/100
964/964 - 1s - loss: 0.4723 - accuracy: 0.7733
Epoch 10/100
964/964 - 1s - loss: 0.4709 - accuracy: 0.7759
Epoch 11/100
964/964 - 1s - loss: 0.4699 - accuracy: 0.7750
Epoch 12/100
964/964 - 2s - loss: 0.4685 - accuracy: 0.7765
Epoch 13/100
964/964 - 1s - loss: 0.4679 - accuracy: 0.7756
Epoch 14/100
964/964 - 1s - loss: 0.4672 - accuracy: 0.7779
Epoch 15/100
964/964 - 1s - loss: 0.4663 - accuracy: 0.7769
Epoch 16/100
964/964 - 1s - loss: 0.4660 - accuracy: 0.7776
Epoch 17/100
964/964 - 1s - loss: 0.4653 - accura

## Compare the models 

In [23]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
Dmodel_loss, Dmodel_accuracy = deep_model.evaluate(X_test_scaled, y_test_categorical, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")
print(f"Deep Neural Network - Loss: {Dmodel_loss}, Accuracy: {Dmodel_accuracy}")

322/322 - 0s - loss: 0.4695 - accuracy: 0.7766
322/322 - 0s - loss: 0.4681 - accuracy: 0.7735
Normal Neural Network - Loss: 0.46954256296157837, Accuracy: 0.776588499546051
Deep Neural Network - Loss: 0.4681321680545807, Accuracy: 0.7734747529029846
